<a href="https://colab.research.google.com/github/drkulkarni236/w266_finalproject/blob/main/m_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install simpletransformers

     |████████████████████████████████| 248 kB 14.1 MB/s 
     |████████████████████████████████| 311 kB 64.5 MB/s 
     |████████████████████████████████| 3.5 MB 57.5 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 6.8 MB 47.5 MB/s 
     |████████████████████████████████| 1.7 MB 56.9 MB/s 
     |████████████████████████████████| 1.2 MB 55.8 MB/s 
     |████████████████████████████████| 9.7 MB 64.0 MB/s 
     |████████████████████████████████| 596 kB 61.5 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 895 kB 69.2 MB/s 
     |████████████████████████████████| 144 kB 61.6 MB/s 
     |████████████████████████████████| 181 kB 65.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 243 kB 62.9 MB/s 
     |████████████████████████████████| 133 kB 58.0 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
     |██████████████

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from tqdm.autonotebook import tqdm
from torch.utils.data import Dataset, DataLoader
from random import shuffle
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score
import logging
ch = logging.StreamHandler(sys.stdout)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

train_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_Korean/train.tsv',
                       sep = '\t', 
                       dtype={'comments':str,'hate':str},
                       nrows = 7897)

print("Dataframe Number of (rows, columns) : {}".format(train_df.shape))

train_df = train_df[['comments','hate']]

pd.options.display.max_colwidth = 200
train_df.head(10)

Mounted at /content/gdrive
Dataframe Number of (rows, columns) : (7896, 4)


,comments,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래야, 공정한 사회지...심은대로 거두거라...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각이 없노,hate
5,10+8 진짜 이승기랑 비교된다,none
6,"100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, 여성 투표권 삭제가 세계의 공통문화로 자리잡을듯. 암탉이 너무 울어댐.",hate
7,10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음,none
8,10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐,offensive
9,10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름기억은해둠ㅇㅇ,offensive


In [ ]:
dev_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_Korean/dev.tsv',
                       sep = '\t', 
                       dtype={'comments':str,'hate':str},
                       nrows = 7897)

print("Dataframe Number of (rows, columns) : {}".format(dev_df.shape))

dev_df = dev_df[['comments','hate']]

pd.options.display.max_colwidth = 200
dev_df.head(10)

Dataframe Number of (rows, columns) : (471, 4)


,comments,hate
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,none
1,지현우 나쁜놈,offensive
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,hate
3,설마 ㅈ 현정 작가 아니지??,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀 하고사시죠.,offensive
5,일베충들 ㅂㄷ거리는것봐라 ㅉㅉ,hate
6,아이즈원 힘내세요...일본 진출도 했으니 일본서 좋은 모습 보여줘도 팬들은 응원 합니다.,none
7,강부자 선생님 전미선 비보에 오열을 하셨다니 눈물이 나네요 힘내세요,none
8,알았어 그만,offensive
9,이영자씨는 진정성 있는거라면 녹화불참으로 끝내지말고 자진하차해라 시청자는 고려도 안하고 일방적 불참은 아닌듯 엠비씨도 시청율 좋아서 고민하는거 같은데 결방할게 아니고 폐지해라,offensive


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_df['hate'] = train_df['hate'].replace(['hate'], 1)
train_df['hate'] = train_df['hate'].replace(['offensive'], 2)
train_df['hate'] = train_df['hate'].replace(['none'], 0)

dev_df['hate'] = dev_df['hate'].replace(['hate'], 1)
dev_df['hate'] = dev_df['hate'].replace(['offensive'], 2)
dev_df['hate'] = dev_df['hate'].replace(['none'], 0)

In [ ]:
train_args = {"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 5}

model = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=3,
    args = train_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
model.train_model(train_df)

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
 
result, model_outputs, wrong_predictions = model.eval_model(dev_df, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/7896 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/987 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/987 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/987 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/471 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

In [ ]:
print(result)

{'mcc': 0.39307584178478017, 'f1': 0.5966029723991507, 'acc': 0.5966029723991507, 'eval_loss': 0.9701858516466819}


#Hindi


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

train_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_Hindi/hindi_dataset.tsv',
                       sep = '\t', 
                       dtype={'comments':str,'hate':str},
                       nrows = 7897)

print("Dataframe Number of (rows, columns) : {}".format(train_df.shape))

train_df = train_df[['text','task_1']]

pd.options.display.max_colwidth = 200
train_df.head(10)

Mounted at /content/gdrive
Dataframe Number of (rows, columns) : (4665, 5)


,text,task_1
0,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर रोका #INDvBAN #CWC19",NOT
1,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांतीदूत के साथ कुछ होगा सब #रंडीरोना शुरू कर देंगे,HOF
2,तुम जैसे हरामियों के लिए बस जूतों की कमी है शुक्र कर अभी तुम्हारी लिंचिंग हुई नहीं है हिंदुओं के जागने की देर है सच में होगी अभी तो तुम जैसे हरामी सुवर ड्रामा बनाएं हो सुवर कहीं का मौलाना। तुम...,HOF
3,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत मिलने की खुशी में एक समर्थक ने इंदौर में हर्ष फायरिंग की #AkashVijayvargiya https://abpnews.abplive.in/india-news/celebratory-firing-outside-bjp-ml...",NOT
4,"चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर्शन, तेजस्वी यादव नदारद #biharencephalitisdeaths https://abpnews.abplive.in/bihar-news/aes-deaths-rjd-protest-in-vidhan-sabha-campus-but-tejashw...",NOT
5,मुंबई में बारिश से लोगों को काफी समस्या हो रही है,NOT
6,Ahmed's dad:-- beta aaj teri mammy kyu nahi baat kr rhi h. Ahmed....,NOT
7,"5 लाख मुसलमान उर्स में, अजमेर की दरगाह पर आते हैं, सिर्फ 300 पुलिस वालों के भरोसे, 2 लाख हिंदुओं की अमरनाथ यात्रा के लिए, 80 हजार कमांडो, पैरामिलिट्री फोर्स, तथा करोड़ों के उपकरण लगाए जा रहे है...",NOT
8,"Do mahashaktiyan mili hain, charo taraf khusi ki leher hai, khus hone wale khus hi rhe hain aur bhakton ko taklif ho rhi hai, khair honi bhi chahiye.",NOT
9,Chants of 'Jai Sri Ram' as Owaisi takes oath: AIMIM chief responds with 'Jai Bhim' @thenewsminute @asadowaisi,NOT


In [ ]:
dev_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_Hindi/hasoc2019_hi_test_gold_2919.tsv',
                       sep = '\t', 
                       dtype={'comments':str,'hate':str},
                       nrows = 7897)

print("Dataframe Number of (rows, columns) : {}".format(dev_df.shape))

dev_df = dev_df[['text','task_1']]

pd.options.display.max_colwidth = 200
dev_df.head(10)

Dataframe Number of (rows, columns) : (1318, 5)


,text,task_1
0,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी धोखा दे सकता है। #IndiaVsPakistan",NOT
1,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश की रक्षा करते #शहीद हुए मां भारती के #सपूतों का ख्याल भी नहीं रखा #दुश्मन से गले मिल गये #शर्मनाक. ये पूरे शहीद का अपमान किया है। मैं बहुत कद्र करते ...,HOF
2,पाकिस्तान को फेकना था फेका गया। जो हार कर भी दुश्मन को हरा दे उसी को हमने नाम दिया। @BCCI ये तुम्हारे समझ के बाहर हैं तुम बॉलीवुड में रंडी रोओ इधर मत घुसो।,HOF
3,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे वो बचाकर रखना क्योंकि यही कल कोई और कल तुम्हारी माँ बहन और बेटी के लिये यूज करेगा तब गुस्सा न करना क्योंकि वो तुम्हारे फेंके शब्द तुम्ही को मारेगा.... ...,NOT
4,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आपका हर लिया गया निर्णय हमे स्वीकार है निवेदन है कि कार्यकर्ताओं पे वरिष्ठ नेता ध्यान दे ये चुनाव भी एक सबक है हमारे लिए कही न कही कार्यकर्ताओं को नजरअ...,NOT
5,@narendramodi @AmitShah @BJP4India @BharatKaPM @TeamDeol #AayegaTohModiHi #BJP4India #BJP #कन्हैया_कुमार ने खोदा कुआँ सिंधिया हारे जुआ शत्रुघ्न खामोश हुआ राज बब्बर बना चूहा 22 राज्य ने 00 छुआ खैर ...,HOF
6,"#कुत्ते भी हो रहे हैं #किडनी और #डायबटीज जैसी #बीमारियों का शिकार, जानें वजह&#8230;",NOT
7,VIVAHIT BaitION KO BHI Patrick KIRSI BHOOMI MAIN BAITON K SAMAN U.P MAIN Adhikar KOUN dilanai KI Kirpa karaiga ? Vicharniy pirsn ? Kya vivahit baition k sath anyay hota rahaiga u.p.main ?,HOF
8,10 एजेंसियों को किसी भी कंप्यूटर की निगरानी और डाटा की जांच का अधिकार दिए जाने के खिलाफ दायर याचिकाओं पर सुप्रीम कोर्ट ने सुनवाई टाली। याचिकाओं में सरकार के आदेश को मनमाना बताया गया है। याचिकाकर्त...,HOF
9,"कश्मीर मे सेना पर पत्थर मारने वाले और पत्थरबाजों के हमदर्द, और, लस्सी न देने पर मर्डर करने वाले लोग पूछ रहे हैं कि बाइक चोर को क्यो पीटा?? तबरेज की पत्नी को मुआवजा और नौकरी देगा दिल्ली वक्फ बोर...",NOT


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_2 = BertModel.from_pretrained("bert-base-multilingual-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_df['task_1'] = train_df['task_1'].replace(['HOF'], 1)
train_df['task_1'] = train_df['task_1'].replace(['NOT'], 0)

dev_df['task_1'] = dev_df['task_1'].replace(['HOF'], 1)
dev_df['task_1'] = dev_df['task_1'].replace(['NOT'], 0)

In [ ]:
train_args = {"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 5}

model_2 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=2,
    args = train_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
model_2.train_model(train_df)

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
 

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4665 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/584 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/584 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/584 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/584 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/584 [00:00<?, ?it/s]

NameError: ignored

In [ ]:
result_2, model_2_outputs, wrong_predictions_2 = model_2.eval_model(dev_df, f1=f1_multiclass, acc=accuracy_score)
print(result_2)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1318 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/165 [00:00<?, ?it/s]

{'mcc': 0.6105704273579913, 'tp': 497, 'tn': 564, 'fp': 149, 'fn': 108, 'auroc': 0.8892237432336885, 'auprc': 0.8775556505635109, 'f1': 0.8050075872534143, 'acc': 0.8050075872534143, 'eval_loss': 0.8391845449607707}
